In [9]:
from api.db import stat_db, crawl_db, CrawlCollections, StatCollections, relation_db, RelationCollections, DESCENDING, ASCENDING
import json
import time
import math
import re
import pandas as pd
import numpy as np
from numpy import nan
from datetime import datetime

In [32]:
bitcoin = crawl_db[CrawlCollections.COINMARKET_HISTORY_PRICE].find_one({'_id': 'bitcoin-cash'})

In [19]:
data = json.loads(bitcoin['data'])

In [20]:
market_cap = data['market_cap_by_available_supply']

In [29]:
for item in market_cap:
    original_date = datetime.fromtimestamp(item[0] / 1000)
    str_time = original_date.strftime('%Y-%m-%d')
    format_date = datetime.strptime(str_time, '%Y-%m-%d').timestamp()
    price_data = stat_db['history_currency_price'].find_one({'_id': format_date})
    if price_data is not None:
        stat_db['history_currency_price'].update_one({'_id': price_data['_id']}, {'$set': {'market_cap.{0}'.format('bitcoin-cash'): item[1]}})
    else:
        stat_db['history_currency_price'].insert_one({'_id': format_date, 'market_cap': {'{0}'.format('bitcoin-cash'): item[1]}})

In [3]:

coinmarket_coins_cursor = crawl_db[CrawlCollections.COINMARKET_CURRENCIES].find()
df = pd.DataFrame([( coins['data']['sort'], coins['_id'], coins['data']['name'], coins['data']['symbol'], coins['data']['url']) for coins in coinmarket_coins_cursor], columns=['sort', 'id', 'name', 'symbol', 'url'])                          
df.url = 'https://coinmarketcap.com' + df.url
writer = pd.ExcelWriter('E:/coinmarket_currency.xlsx')
df = df.sort_values(by='sort')
df.to_excel(writer, sheet_name='Sheet1', index=False)
writer.save()


In [ ]:
# 最近一次分析的日期有多少个货币
def how_many_coins_in_latest_stat_date():
    currency_history_cursor = stat_db[StatCollections.HISTORY_CURRENCY_PRICE_STAT].find().sort([('_id', DESCENDING)]).limit(1)
    latest_time_stat = currency_history_cursor[1]
    price_usd = latest_time_stat['price_usd']
    coins = list(price_usd.keys())
    print(len(coins))

## 分析某个货币的排名随时间变化数据


In [59]:
try:
    d = stat_db['history_currency_rank'].find_one({'currency_id': 'bitcoin'})
    latest = max(d['rank'], key=lambda x: x[0])
except TypeError as e:
    latest = 0
history_data = stat_db[StatCollections.HISTORY_CURRENCY_PRICE_STAT].find({'_id': {'$gt': latest}}).sort([('_id', ASCENDING)])
count = len(list(history_data))
for index,data in enumerate(history_data):
    if index / 100 == 0:
        print(str(index) + '/' + str(count))
    t = data['_id'] 
    prices = data['price_usd']
    sl = sorted([[k, prices[k]] for k in list(prices.keys())], key=lambda x: x[1], reverse=True)
    for index, item in enumerate(sl):
        stat_db['history_currency_rank'].find_one_and_update(
            {'currency_id': item[0]},
            {'$push': {'rank': [t, index, len(sl)]}},
            upsert=True)    

In [ ]:

currency = crawl_db[CrawlCollections.COINMARKET_HISTORY_PRICE].find_one({'_id': 'acoin'})
price_data = json.loads(currency['data'])
if 'last_resample_time' in currency:
    last_resample_time = 0
else:
    last_resample_time = 0
data = price_data['market_cap_by_available_supply']

arr = []
if last_resample_time == 0:
    t = time.time() * 1000 - 24 * 3600 * 30 * 1000
else:
    t = last_resample_time
for item in reversed(data):
    if item[0] > t:
        arr.append(item)
    else:
        break     
price_df = pd.DataFrame(arr, columns=['time', 'price'])
print(price_df)
price_df.time = price_df['time'].apply(lambda x: datetime.fromtimestamp(x / 1000))
price_df.set_index('time', inplace=True)
price_df.values
price_df = price_df.resample('D').mean()
for index, item in price_df.iterrows():
    print(index, item['price'])
for index, row in price_df.iterrows():
    print(index.timestamp(), row['price'])


In [16]:
market_cap = {'bee-token': 18490564.916666668, 'viacoin': 58021407.0, 'prcoin': 18205.0, 'supercoin': 415432.0, 'blockport': 19558687.833333332, 'eaglecoin': 115532.75, 'lykke': 18312549.0, 'the-cypherfunks': 759369.0, 'zcoin': 219470321.0, 'deviantcoin': 5052992.364583333, 'bitcoin-gold': 1493822758, 'richcoin': 0.0, 'bitstation': 0.0, 'blackmoon': 17249368, 'syncfab': 0.0, 'sentinel': 25663423.864583332, 'ckusd': 0.0, 'lightning-bitcoin': 0.0, 'bitcoinx': 0, 'ebitcoin-cash': 551699.0, 'tether': 2280297688.0, 'presearch': 42672585.0, 'capricoin': 2634958, 'genaro-network': 111287437, 'cryptcoin': 0, 'elixir': 10652546.0, 'wa-space': 0.0, 'trinity-network-credit': 42505082.583333336, 'dao-casino': 4954377, 'cryptonex': 299978965, 'lazaruscoin': 0, 'verify': 3493073.8333333335, 'ignis': 86139865.41666667, 'lendroid-support-token': 0.0, 'primecoin': 19517695.0, 'decred': 559593845.0, 'coin2-1': 212365, 'monero-classic': 0.0, 'ulatech': 5422.0, 'women': 16909.5, 'mybit-token': 10435943, 'blackstar': 57505, 'posex': 23263.0, 'leocoin': 22821005.0, 'bitusd': 13243820, 'project-x': 11393.0, 'chesscoin': 324326.0, 'electroneum': 175038296, 'marijuanacoin': 86485.0, 'ethereum-lite': 0, 'masternodecoin': 2983414.5, 'ip-exchange': 9112652.0, 'accelerator-network': 752204.5833333334, 'ox-fina': 0, 'billionaire-token': 588838, 'roofs': 103756.5, 'save-and-gain': 32266.0, 'synergy': 1518995.0, 'waykichain': 0.0, 'havven': 34155220.958333336, 'obits': 7686510, 'icon': 1557018559.5, 'noah-coin': 0.0, 'levoplus': 3099.0, 'earth-token': 0.0, '300-token': 295741.0, 'nyancoin': 547023.0, 'high-voltage': 389561, 'viberate': 33771453.0, 'kobocoin': 1992892.0, 'smartcoin': 550267.0, 'elcoin-el': 376207, 'rcoin': 0.0, 'bitair': 567544, 'monero': 4412640180.0, 'swarm-city': 15599609.0, 'topcoin': 0.0, 'qubitcoin': 602252.0, 'jingtum-tech': 0.0, 'phoenixcoin': 552855.0, 'unobtanium': 27566952.0, 'uncoin': 0.0, 'popularcoin': 1258770.0, 'bata': 1180415, 'thekey': 85834462.5, 'clubcoin': 0, 'glasscoin': 226057, 'blockmason': 43269251.5, 'authorship': 948350.25, 'bnktothefuture': 0.0, 'iot-chain': 69426186.08333333, 'aditus': 0.0, 'quantum': 10789272.75, 'bitbar': 645645, 'melon': 45882331.0, 'secretcoin': 144975.0, 'bulwark': 10119855.666666666, 'freicoin': 269768.0, 'neverdie': 1129057.5, 'bitcoin-scrypt': 611872, 'benjirolls': 94785, 'fujicoin': 1111579.0, 'vericoin': 20539238.0, 'oax': 19105246.75, 'waves': 535386999.0, 'chronos': 201302.0, 'royal-kingdom-coin': 46978.5, 'vtorrent': 5060281.0, 'inpay': 2178660.0, 'magnum': 7285.0, 'machinecoin': 987373.0, 'populous': 939275064.5, 'hi-mutual-society': 27893116.666666668, 'pizzacoin': 2721.0, 'mobius': 46145503.0, 'bipcoin': 73311, 'filecoin': 0, 'eurocoin': 125754, 'jet8': 6092664.583333333, 'gnosis-gno': 113433660.0, 'ontology': 546330014.375, 'russian-mining-coin': 8046204.25, 'sphre-air': 0.0, 'wagerr': 51581706.75, 'credo': 12443169, 'monkey-project': 5042976.166666667, 'irishcoin': 428644.0, 'espers': 8359159.0, 'piecoin': 41451, 'trollcoin': 461547.0, 'vechain': 2126104928.75, 'the-champcoin': 10497621.0, 'spectrecoin': 12755064.0, 'groestlcoin': 102741732.0, 'bowhead': 1315440, 'c20': 70792104.75, 'cryptopiafeeshares': 0, 'paypex': 129246204, 'prochain': 18363654.666666668, 'monetha': 27400979.25, 'nevacoin': 302433.0, 'prototanium': 391561.0, 'advanced-internet-blocks': 239739, 'bitcoin-red': 395923.0, 'artex-coin': 48773, 'herocoin': 6455258.083333333, 'ellaism': 2472799, 'huncoin': 0.0, 'bezop': 3375393, 'force': 2687991, 'bitdegree': 17710086.916666668, 'magiccoin': 964508, 'pioneer-coin': 113697.08333333333, 'leadcoin': 11577328.875, 'amlt': 0.0, 'indorse-token': 2314949, 'zengold': 0.0, 'datum': 43876628, 'education-ecosystem': 5402715.958333333, 'bit20': 827417.5, 'litecred': 36646.0, 'sparks': 0.0, 'acchain': 0, 'litecoin': 8556188674.0, 'nebulas-token': 248147130, '808coin': 2875091.0, 'pulse': 43940.0, 'xpa': 13165076.5, 'united-bitcoin': 0.0, 'envion': 50850484, 'stex': 0.0, 'safe-exchange-coin': 24493460.0, 'mercury': 30118400.0, 'digital-rupees': 195195, 'eztoken': 4621765.5, 'decent-bet': 13801202.5, 'spectre-utility': 7493703.666666667, 'karmacoin': nan, 'sirin-labs-token': 94164296.08333333, 'influxcoin': 289289.0, 'selfkey': 25900547.583333332, 'klondikecoin': 0.0, 'interstellar-holdings': 1843234.5, 'denarius-dnr': 3662930.5, 'etherdelta-token': 0.0, 'sooncoin': 0.0, 'sovereign-hero': 897469.25, 'monetaryunit': 17499514.0, 'mark-space': 0.0, 'suretly': 793193, 'cheapcoin': 0, 'toacoin': 9834907.0, 'adtoken': 39288510.0, 'auroracoin': 12375511, 'monacoin': 263126487.0, 'gridcoin': 22932201, 'salt': 223813292.5, 'bitgem': 679771, 'clearpoll': 9557612.25, 'obsidian': 3821056.25, 'terranova': 0.0, 'diamond': 23488470, 'expanse': 18575683, 'yoyow': 34688311.0, 'adzcoin': 1553219, 'trueusd': 11331432.041666666, 'dix-asset': 227556, 'flaxscript': 84021.0, 'rchain': 498600024.75, 'magi': 3689101.0, 'cloakcoin': 76867930, 'bitgold': 301067, 'qvolta': 2982822.25, 'tattoocoin-limited': 0.0, 'idex-membership': 6122700, 'content-and-ad-network': 0.0, 'elysium': 17277, 'beatcoin': 101278, 'gamechain': 0.0, 'breakout': 5266085.0, 'slothcoin': 0.0, 'zcash-gold': 224413.75, 'tidex-token': 1596346.5, 'cubits': 0, 'bitcoin': 160768310895, 'experty': 5479567, 'universal-currency': 6129134.0, 'bridgecoin': 47960370, 'pure': 10944045.5, 'sjwcoin': 0.0, 'pirl': 7784761.75, 'kronecoin': 76985.5, 'galactrum': 1328036.4166666667, 'chancoin': 310040, 'crown': 32293544, 'metaverse': 41125516.5, 'fargocoin': 0, 'digital-credits': 2207, 'shift': 35981999.0, 'brahmaos': 0.0, 'antimatter': 0, 'datx': 0.0, 'kurrent': 631781.0, 'xinfin-network': 22502072.083333332, 'avoncoin': 0, 'bitcoal': 119353, 'marxcoin': 0.0, 'linkedcoin': 0.0, 'globalboost-y': 171183.0, 'ponzicoin': nan, 'biteur': 133782, 'unikoin-gold': 39481891.75, 'pillar': 85933319.75, 'polis': 11592282.5, 'komodo': 371982119.0, 'argus': 21634, 'aelf': 304562500, 'polcoin': 183875.0, 'bitshares-music': 10217879, 'bitcurrency': 296141, 'gulden': 46983577, 'fundyourselfnow': 2216082, 'wax': 208473682.33333334, 'bumbacoin': 136323, 'emphy': 1331886.0, 'californium': 88121, 'remme': 11330869.916666666, 'hellogold': 6954951.25, 'dignity': 0.0, 'mcap': 1518765.25, 'tigercoin': 374465.0, 'fusion': 158296148, 'regacoin': 0, 'steem': 910011736.0, 'kayicoin': 190612.0, 'autonio': 6551771.75, 'stealthcoin': 10329963.0, 'biblepay': 1991210, 'crevacoin': 13945, 'bitcoin-planet': 209213.25, 'fincoin': 167031, 'px': 228297.0, 'octanox': 305972, 'aeon': 31079086, 'bread': 58826495, 'iconic': 54324.0, 'polyswarm': 17762122.875, 'catcoin': 203282, 'xtd-coin': 0.0, 'swissborg': 19014107.416666668, 'arbit': 46441, 'caliphcoin': 1885, 'tigereum': 677484.25, 'e-gulden': 3286340, 'electra': 40639299.5, 'bitbtc': 601617, 'dalecoin': 115665.75, 'republic-protocol': 33477286.375, 'ultracoin': 546880.0, 'neo-gold': 0.0, 'fapcoin': 0, 'nanjcoin': 37003539.25, 'whitecoin': 14105212.0, 'nucleus-vision': 177263839.29166666, 'rawcoin2': 20179.0, 'bitserial': 0.0, 'todaycoin': 0.0, 'bytecoin-bcn': 976673649, 'stellar': 6317728777.0, 'coimatic-3': 17329, 'lightchain': 0.0, 'humaniq': 31115454.0, 'altcommunity-coin': 0, 'linkeye': 40394340, 'virtacoinplus': 34085.0, 'hackspace-capital': 8263096.5, 'wawllet': 0.0, 'solaris': 12101231.0, 'bitclave': 19456250.083333332, 'request-network': 170632302.5, 'musiconomi': 0.0, 'airswap': 87585337.5, 'bottos': 62255961.5, 'gxchain': 216729600.0, 'universe': 1300053.0, 'stealth': 10329963.0, 'neutron': 5213326.0, 'akuya-coin': 0, 'desire': 129321, 'bitwhite': 1603778.5833333333, 'primas': 12144229.75, 'levocoin': 0.0, 'monaco': 197929475.25, 'stipend': 4048630.2083333335, 'darcrus': 2793492, 'firstblood': 34305826, 'postcoin': 323239.0, 'aware': 0.0, 'dutch-coin': 0, 'electronic-pk-chain': 0.0, 'modum': 52209822.25, 'redcoin': 210106.0, 'adshares': 5017424, 'deltacredits': 0, 'peerplays-ppy': 21913708.0, 'rebl': 7970351.0, 'fuzzballs': 116051.0, 'icoin': 283631.0, 'dew': 102553433, 'bankex': 33510696.833333332, 'canada-ecoin': 2096265, 'cpchain': 35734008.166666664, 'smartlands': 11556248.583333334, 'sentinel-chain': 25974773.166666668, 'berncash': 284037, 'bowscoin': 77787, 'cabbage': 220798, 'atbcoin': 8618400, 'measurable-data-token': 21679524.833333332, 'ripio-credit-network': 72115804.25, 'babb': 39258285.666666664, 'unicoin': 1316974.0, 'propy': 34658002.75, '8bit': 405485, 'evergreencoin': 3664836.0, 'waltonchain': 442044553.0, 'cvcoin': 11751909, 'kingn-coin': 47054.0, 'ride-my-car': 77899.0, 'auctus': 13217506.729166666, 'yocoin': 1921895.0, 'blazercoin': 0.0, 'effect-ai': 14173633.177083334, 'bitsum': 524170.71875, 'rise': 13624203.0, 'ins-ecosystem': 58310454.916666664, 'scorecoin': 361425.0, 'magnet': 2706400.8333333335, 'tenx': 169328812.25, 'qube': 0.0, 'halalchain': 0.0, 'peercoin': 58423247.0, 'upfiring': 3299994.0, 'eryllium': 98957, 'satoshimadness': 187761.0, 'blackcoin': 23731231, 'karbo': 3714968.0, 'anarchistsprime': 26117, 'tagcoin': 582320.0, 'latoken': 19797289.0, 'royalties': 0, 'zcash': 1168845097.0, 'eternity': 317701, 'tokes': 9451199.0, 'quebecoin': 249152.0, 'trust': 30022392.0, 'dinastycoin': 1051801.5, 'novacoin': 8656240.0, 'procurrency': 1315260.0, 'cryptojacks': 460402, 'master-swiscoin': 53869.0, 'arbitracoin': 0.0, 'startcoin': 822954.0, 'rsgpcoin': 45618.25, 'coss': 49059281, 'bloomtoken': 22449944.416666668, 'te-food': 17402134.75, 'paragon': 17700698.25, 'ea-coin': 0, 'moac': 0.0, 'incakoin': 1325507, 'b2bx': 8639071.25, 'bitstar': 873011, 'bonpay': 5933230.666666667, 'veros': 37047.0, 'ethereum-blue': 8047861.75, 'radium': 19933775.0, 'ripto-bux': 34095.0, 'bolivarcoin': 296248, 'poa-network': 123776180.125, 'blitzcash': 4078511, 'moin': 1517921, 'europecoin': 6143624.0, 'rabbitcoin': 0.0, 'neoscoin': 13457395.0, 'gold-pressed-latinum': 67962, 'bios-crypto': 40262, 'tajcoin': 162311.0, 'newbium': 150051.0, 'worldcore': 12565768.416666666, 'aerium': 320298, 'telcoin': 69924300.58333333, 'pura': 55269434.0, 'dmarket': 29964427.166666668, 'zilla': 9417191.083333334, 'alpacoin': 0, 'dfscoin': 996100, 'salus': 53356892.0, 'universa': 41922407.916666664, 'gameunits': 167934, 'aigang': 6040405.166666667, 'indicoin': 0, 'grandcoin': 0.0, 'adbank': 10923379.916666666, 'qunqun': 35064322.416666664, 'cryptoping': 6862084, 'golem-network-tokens': 512891763.0, 'blazecoin': 620430, 'ethlend': 84748919.08333333, 'wanchain': 786183715.4583334, 'quantstamp': 121088592.0, 'ganjacoin': 1802008, 'hempcoin': 27681190.0, 'matrix-ai-network': 153193500, 'britcoin': 782410, 'runners': 0.0, 'odyssey': 52631150.0, 'flik': 1901880.0, 'newyorkcoin': 11954551.0, 'flixxo': 13746998.25, 'lendconnect': 1819815.0, 'vsync-vsx': 2425970.25, 'techshares': 0.0, 'stk': 29673323.083333332, 'motocoin': 367376.0, 'supernet-unity': 0.0, 'orbitcoin': 2047627.0, 'straks': 2085386.75, 'global-currency-reserve': 4574294, 'ucom': 2665168.0, 'drp-utility': 1134920.25, 'stellite': 0.0, 'coupecoin': 10546.25, 'bismuth': 17198869, 'dopecoin': 8283251.0, 'tokenclub': 16697366.583333334, 'bitsoar': 0, 'nodecoin': 10929.0, 'investdigital': 0.0, 'ucash': 9055784.25, 'soarcoin': 43030303.0, 'grimcoin': 187889, 'lala-world': 11885014.25, 'boostcoin': 188062, 'crtcoin': 0, 'guaranteed-ethurance-token-extra': 10165652.5, 'vivo': 1464838.75, 'solarflarecoin': 149678.0, 'bluzelle': 90797107.5, 'lockchain': 11745719.916666666, 'ccore': 203556, 'eltcoin': 1483145.75, 'colossusxt': 18053152.0, 'numus': 1472920.4166666667, 'crypterium': 75558883.33333333, 'syscoin': 288555941.0, 'swisscoin': 0.0, 'chainlink': 182722400, 'gamecredits': 122548679.0, 'metalcoin': 0.0, 'tokendesk': 0.0, 'bitconnect': 13437446, 'astro': 6566625, 'opescoin': 0.0, 'bitqy': 10062429, 'miners-reward-token': 1389512.5, 'lampix': 4540790.5, 'aion': 518357614.25, 'paycon': 177882.0, 'mywish': 4328011.416666667, 'sakuracoin': 0.0, 'lomocoin': 17386478.0, 'reecoin': 201830.0, 'enigma-project': 186199689.5, 'creativecoin': 2789624.0, 'copico': 596833, 'polymath-network': 120762873.66666667, 'fantomcoin': 1031401.0, 'atlant': 9346789.75, 'first-bitcoin-capital': 0, 'bastonet': 0, 'hubii-network': 7764438.25, 'html-coin': 29532861.666666668, 'evotion': 149316.0, 'ethereum': 68228258883, 'bitcoin-private': 1019701467.5, 'mustangcoin': 175519.0, 'shadowcash': 1029965.0, 'omni': 19429410.0, 'tokugawa': 414239.4166666667, 'campuscoin': 3353386, 'red-pulse': 97101259.5, 'bancor': 247865949, 'x-coin': 104041.0, 'aurora-dao': 20543494.0, 'dropil': 69931416.875, 'hirematch': 3310640.2083333335, 'fidelium': 0.0, 'valuechain': 0.0, 'candy': 0.0, 'goldblocks': 415951, 'zilbercoin': 0.0, 'pokecoin': 0.0, 'hive-project': 22843443.5, 'dragonchain': 309186366.0, 'megacoin': 1009592.0, 'cortex': 195812870.125, 'library-credit': 52835836, 'virta-unique-coin': 130750.5, 'pesetacoin': 7083423.0, 'unitus': 3158314.0, 'open-trading-network': 10788767.75, 'insights-network': 9867190.666666666, 'iquant': 0.0, 'kyber-network': 273382555.75, 'bytom': 1045321830, 'pakcoin': 411494.0, 'playkey': 8356840.166666667, 'tokenpay': 101976932.08333333, 'geocoin': 7898159.0, 'bitcoindark': 130097748, 'jewels': 54930, 'apx': 7008177, 'elastos': 220324762.75, 'dnotes': 3236261, 'sportscoin': 0.0, 'domraider': 14274920.5, 'wepower': 62610245.333333336, 'virtacoin': 525738.0, 'compucoin': 150087, 'dubaicoin-dbix': 10694166.0, 'francs': 0.0, 'monster-byte': 855761.75, 'cargox': 14090766.083333334, 'ammo-reloaded': 261858, 'sydpak': 31419.0, 'ergo': 2444452.5, 'cindicator': 154657607.0, 'huobi-token': 0.0, 'aurumcoin': 2328262, 'synereo': 31754075.0, 'netbit': 0.0, 'veriumreserve': 4890266.0, 'rialto': 14752117.75, 'india-coin': 0.0, 'ibank': 14840.0, 'deutsche-emark': 1291519, 'royalcoin': nan, 'snipcoin': 0.0, 'omisego': 1742146974.75, 'jetcoin': 759153.5, 'medical-chain': 26097317.75, 'blocknet': 105686369.0, 'spacechain': 24325306.583333332, 'money': 110979.0, 'xaurum': 9678891.0, 'kolion': 1034813.75, 'playercoin': 36186.75, 'encryptotel-eth': 0.0, 'okcash': 13859075.0, 'aricoin': 602611, 'oraclechain': 19986127.5, 'equitrader': 3254926.75, 'bitshares': 778835098, 'tiesdb': 6291016.75, 'anoncoin': 1836798, 'credits': 100337766.95833333, 'bunnycoin': 5110974, 'trumpcoin': 573460.0, 'primalbase': 4150287.25, 'brat': 176264, 'heat-ledger': 8563411.0, 'ardor': 378614814, 'qora': 0.0, 'ethbits': 1697046, 'dollar-online': 25813, 'mazacoin': 1322094.0, 'numeraire': 25723277, 'sophiatx': 96670523.41666667, 'steneum-coin': 423714.8333333333, 'songcoin': 76634.0, 'litecoin-plus': 1017638.0, 'goldcoin': 6782260, 'hyper': 0.0, 'yashcoin': 1547537.0, 'canyacoin': 12723829.583333334, 'breakout-stake': 8375411.0, 'darklisk': 0, 'postoken': 58063.0, 'ethereum-gold': 334931.25, 'concoin': 11589.0, 'adx-net': 82537576, 'fedoracoin': 7021122, 'hedge': 8104738.25, 'warp': 63441.0, 'bitbase': 0, 'fluz-fluz': 9822833.791666666, 'fantasy-cash': 1775135.5, 'platinumbar': 1233100.0, 'time-new-bank': 122106081.41666667, 'commodity-ad-network': 1111699.8333333333, 'databits': 14878795, 'cryptocarbon': 649540, 'exchangen': 59945.5, 'student-coin': 1230485.25, 'gaycoin': 0.0, 'creatio': 124143, 'yee': 24562560.333333332, 'compcoin': 0, 'bitsilver': 529356, 'gameleaguecoin': 0.0, 'digital-bullion-gold': 0, 'starcointv': 0.0, 'minereum': 655515.0, 'rlc': 119182973.0, 'eboostcoin': 8997660.0, 'ofcoin': 0.0, 'titanium-bar': 0, 'zrcoin': 3202088.25, 'trade-token': 25866724.833333332, 'nvo': 7616474.75, 'edgeless': 59431958, 'neumark': 13133431.333333334, 'myriad': 16294087.0, 'emerald': 278102, 'allion': 105085, 'unify': 1292272.75, 'flypme': 1777004.25, 'banca': 24381022.583333332, 'etheroll': 13176423, 'nework': 6450597.916666667, 'oyster': 59858431.5, 'echolink': 18096921.166666668, 'trueflip': 6236859, 'eidoo': 69705994.0, 'bitcoin-diamond': 707092689.0, 'qtum': 1887137560.5, 'erc20': 679778.0, 'yenten': 101974.66666666667, 'whalecoin': 390075.5, 'zoin': 23145244.0, 'ereal': nan, 'fucktoken': 915116.0, 'bitasean': 81990.5, 'nuls': 119626599.75, 'harmonycoin-hmc': 4287.5, 'shilling': 74885.0, 'zephyr': 3733499.0, 'prismchain': 0.0, 'geertcoin': 32815, 'nullex': 2042192.0, 'revolutionvr': 18978981.0, 'weth': 0.0, 'gifto': 119242618.75, 'high-gain': 0, 'firstcoin': 1870931.0, 'coinpoker': 12104223.708333334, 'hydro-protocol': 18837413.75, 'blitzpredict': 2560823.375, 'agrolifecoin': 137395, 'draftcoin': 1068399, 'swagbucks': 576898.0, 'dogecoin': 624706694, 'creditbit': 4667203, 'ravencoin': 31246109.166666668, 'internxt': 7318392.0, 'plexcoin': 0.0, 'sharechain': 3639995.5, 'posw-coin': 19372468.0, 'origintrail': 50143766.75, 'alqo': 13941299, 'honey': 159578.0, 'cobinhood': 37787094, 'ur': 0.0, 'icobid': 166071.0, 'pascal-lite': 277658.0, 'tezos': 0.0, 'singulardtv': 63474000.0, 'bolenum': 111821.25, 'smoke': 0.0, 'soma': 737358.6666666666, 'moneta2': 0.0, 'biocoin': 4208710, 't-coin': 0.0, 'trident': 165853.0, 'sagacoin': 2074269.0833333333, 'feathercoin': 52975593, 'oxycoin': 12307392.0, 'equal': 792575.1666666666, 'starcredits': 1137620.0, 'payfair': 11181556.75, 'gcn-coin': 10322000, 'etherparty': 48335715.0, 'tattoocoin': 415022.0, 'nushares': 776072.0, 'sonm': 76181619.0, 'bitdeal': 511239.75, 'roulettetoken': 356002.75, 'namecoin': 29785506.0, 'dashcoin': 530911, 'qbic': 873467.25, 'harvest-masternode-coin': 0.0, '1337coin': 4566276, 'qash': 286480250.0, 'leacoin': 256750.0, 'latiumx': 1665000.0, 'fairgame': 0.0, 'votecoin': 484142.3333333333, 'wabi': 60598917.666666664, 'asiadigicoin': 78983, 'garlicoin': 564621.1666666666, 'swftcoin': 28456312.333333332, 'oneroot-network': 21383865.166666668, 'macron': 416739.0, 'restart-energy-mwat': 22487848.625, 'halloween-coin': 0, 'theta-token': 89368109.66666667, 'litedoge': 2301603.0, 'pandacoin-pnd': 8810436.0, 'macro1': 0.0, 'elacoin': 157139, 'gladius-token': 10202972, 'aeron': 22181867.5, 'ecobit': 9301532.75, 'snovio': 17086499.25, 'opus': 6324860, 'bitmark': 5898278, 'true-usd': 11331432.041666666, 'aventus': 9657345.0, 'ambercoin': 370840, 'starta': 4055449.6666666665, 'impact': 248818.0, 'byteball': 176113950, 'selfiecoin': 10300.0, 'abjcoin': 198361, 'fujinto': 362466, 'zeitcoin': 6592777.0, 'flycoin': 463955, 'alis': 13804032.5, 'zeusshield': 26587043.25, 'selfsell': 0.0, 'metal': 120975523.25, 'dentacoin': 315716336, 'unlimitedip': 0.0, 'fluttercoin': 1559805, 'pirate-blocks': 0, 'cyder': 0, 'storj': 154642995.75, 'aeternity': 603742062, 'rupaya': 3404218.5, 'cybermiles': 103817726.66666667, 'shipchain': 33344627.583333332, 'wild-beast-block': 29901.0, 'dero': 0, 'centrality': 338050633.4166667, 'jibrel-network': 56884637.5, 'vip-tokens': 71966.0, 'grid': 24001844.0, 'digital-money-bits': 241748, 'speed-mining-service': 2921192, 'aquariuscoin': 1482109, 'cfun': 0.0, 'adhive': 0.0, 'impulsecoin': 29643, 'factom': 261473302, 'zeepin': 37099306.916666664, 'shorty': 1510080.0, 'karma': 21830112.333333332, 'trackr': 336712.25, 'stox': 16695333.5, 'change': 8384592.75, 'zayedcoin': 62018.0, 'u-network': 0.0, 'refereum': 33162930.333333332, 'show': 0.0, 'ignition': 1832434.6666666667, 'molecular-future': 0.0, 'trickycoin': 0.0, 'funcoin': 91569, 'sprouts': 4990402.0, 'poly-ai': 518.5833333333334, '42-coin': 1443979, 'destiny': 31707, 'cannation': 85286.0, 'pylon-network': 3247538.25, 'digicube': 229967, 'valorbit': 553365.0, 'ethorse': 6775796.0, 'syndicate': 10534454, 'dimecoin': 29517379, 'bitboost': 1853945, 'skycoin': 157061994.0, 'guccionecoin': 117526, 'golfcoin': 235124, 'aragon': 108454596, 'infinity-economics': 0.0, 'stratis': 550611008.0, 'hodlcoin': 599858, 'ethereum-classic': 2183670812, 'credence-coin': 42149, 'blockpay': 416100, 'paccoin': 25991132.0, 'coinlancer': 1978123.4166666667, 'unbreakablecoin': 1948647, 'elementrem': 504796, 'hodl-bucks': 0.0, 'vulcano': 0.0, 'ultimate-secure-cash': 692085.0, 'halcyon': 489998, 'dragon-coins': 0.0, 'dollarcoin': 212669, 'bitcny': 36151602, 'octocoin': 265357.0, 'rubycoin': 15004925.0, 'cannabiscoin': 5001359, 'bluecoin': 1819103, 'internet-of-people': 9146954, 'homeblockcoin': 88705, 'riecoin': 4117930.0, 'bitcoin-atom': 19437311.916666668, 'crycash': 1958514.78125, 'consensus': 0.0, 'shadow-token': 341384.75, 'edrcoin': 0, 'cartaxi-token': 303707.0, 'intensecoin': 2526924.25, 'guncoin': 1015019, 'florincoin': 16598021, 'greenmed': 0.0, 'pundi-x': 0.0, 'bitcoinfast': 322900, 'neuro': 387275.75, 'cypher': 260899, 'cryptopay': 5147587, 'mercury-protocol': 2760143.5, 'appcoins': 58097886.333333336, 'geysercoin': 9828.75, 'philosopher-stones': 332280.0, 'bitok': 0, 'opal': 760208.0, 'budbo': 0.0, 'vault-coin': 5534.0, 'cybervein': 0.0, 'gems-protocol': 0.0, 'legolas-exchange': 36514334.208333336, 'corion': 0.0, 'titanium-blockchain': 0.0, 'cryptoworldx-token': 33260, 'particl': 157828607, 'eosdac': 0.0, 'argentum': 2446415, 'bitradio': 747855, 'purevidz': 440185.0, 'jin-coin': 184778.0, 'vaperscoin': 75425.0, 'zsecoin': 0.0, 'potcoin': 30048554.0, 'bitz': 348696, 'luna-coin': 48584.0, 'daxxcoin': 440159, 'terracoin': 3520165.0, 'e-dinar-coin': 19604692, 'viuly': 8136949.75, 'nav-coin': 94210965.0, 'truckcoin': 431625.0, 'bitdice': 7849240, 'mixin': 453840080.5, 'dynamiccoin': 0, 'madcoin': 75795.0, 'emercoin': 142843144, 'hacken': 18834055.0, 'uquid-coin': 19602916.666666668, 'zennies': 1118558.0, 'faircoin': 28377047, 'doubloon': 48559, 'litebar': 213685, 'mysterium': 10803678.25, 'bitzeny': 2807740, 'goldmaxcoin': 0.0, 'ethersportz': 2327172, 'foldingcoin': 14272811, 'bean-cash': 26915076, 'korecoin': 6819731.0, 'eventchain': 3102660.6666666665, 'pinkcoin': 9050386, 'rublebit': 0.0, 'shellcoin': 0, 'sisa': 0.0, 'msd': 0.0, 'attention-token-of-media': 22882700, 'embercoin': 0, 'peepcoin': 1039093.0, 'indahash': 19991313.333333332, 'bankcoin': 239154, 'gas': 309077494, 'joincoin': 94062.0, 'quatloo': 181612.0, 'iota': 6032942683, 'blockpool': 4064064, 'dai': 27488198, 'fidentiax': 10501207.333333334, 'sojourn': 117993.2, 'securecoin': 329861.0, 'starchain': 0.0, 'smartmesh': 131048686.08333333, 'litecoin-ultra': 44896.0, 'bitcoin-cash': 25104362400, 'safecoin': 0.0, 'skeincoin': 1357463.0, 'evil-coin': 378902, 'cannacoin': 300518, 'netko': 1152076.0, 'rupee': 3329365.0, 'lepen': 0.0, 'haven-protocol': 0, 'chips': 7507405, 'stronghands': 207239.0, 'tokenomy': 36486966.083333336, 'hyper-pay': 0.0, 'condensate': 2377479.0, 'briacoin': 22548, 'guess': 485400, 'boscoin': 147427641, 'neurotoken': 0.0, 'titcoin': 624725.0, 'photon': 835771.0, 'musicoin': 13829044.0, 'transfercoin': 10906276.0, 'etheriya': 831793, 'deeponion': 33557139, 'iungo': 5323289.916666667, 'odem': 26293914.0, 'mooncoin': 10680356.0, 'civic': 143466885, 'qwark': 5913890.0, 'vslice': 3772124, 'hat-exchange': 13140283.0, 'guppy': 37209225.0, 'fazzcoin': 0.0, 'ixledger': 19335110.25, 'quazarcoin': 64841, 'mothership': 22569995.0, 'devery': 18062920.833333332, 'landcoin': 0.0, 'maecenas': 12144510.0, 'cachecoin': 89222, 'visio': 349672.0, 'tittiecoin': 684509.0, 'amber': 92362653.75, 'lamden': 34905510.916666664, 'javascript-token': 33970.75, 'everex': 27962343.75, 'belacoin': 6684331, 'bytecent': 1507101.0, 'verge': 912154346.0, 'ace': 3478559, 'growers-international': 1102876, 'profile-utility-token': 9823808.833333334, 'buzzcoin': 3650034, 'bodhi': 24771189.916666668, 'aichain': 26324679.583333332, 'digitalprice': 2530656, 'debitcoin': 119758.0, 'data': 90865813.08333333, 'woodcoin': 1224466.0, 'pareto-network': 17977392.083333332, 'triangles': 920758, 'acute-angle-cloud': 0, 'imbrex': 2799320, 'bt2-cst': 0, 'napoleonx': 12317585.916666666, 'ruff': 83740232.66666667, 'travelflex': 9278682, 'safe-trade-coin': 0.0, 'memetic': 6432636.0, 'joulecoin': 426419.0, 'etherecash': 0.0, 'lanacoin': 769330.0, 'xtrabytes': 40444639.0, 'digixdao': 572980000, 'e-coin': 4404696, 'goldpieces': 103066, 'publica': 10402892.333333334, 'circuits-of-value': 9731590, 'loom-network': 143565255.70833334, 'printerium': 66293.0, 'bestchain': 0, 'qlink': 52482640.0, 'deepbrain-chain': 157881000, 'steps': nan, 'triaconta': 1980506.111111111, 'genesis-vision': 86018761.25, 'dynamic-trading-rights': 121622576.66666667, 'ink': 75505226.33333333, 'coinonatx': 319207, 'cappasity': 23734326.0, 'incent': 17603835.0, 'paycoin2': 524862, 'poet': 134790042, 'iht-real-estate-protocol': 64056530.958333336, 'amsterdamcoin': 181864, 'proud-money': 0.0, 'starcash-network': 95940.5, 'parallelcoin': 161113, 'coffeecoin': 0, 'ethbet': 593631.75, 'pepe-cash': 27730103.0, 'spreadcoin': 5461808.0, 'exrnchain': 8172576.0, 'friends': 0.0, 'plncoin': 24486.0, 'comsa-eth': 0, 'monacocoin': 4314840.5, 'sugar-exchange': 617360.9166666666, 'maverick-chain': 3723287.1666666665, 'dether': 7971049.75, 'exchange-union': 0.0, 'eccoin': 29762499, 'tokyo': 305535.75, 'airtoken': 9976128.5, 'bitcloud': 4271108, 'cashaa': 14003483, 'gaia': 551290, 'protean': 0.0, 'divi': 17997048.0, 'useless-ethereum-token': 60850, 'encryptotel': 3969984, 'sativacoin': 175568.0, 'kickico': 40197840.5, 'noblecoin': 2561865.0, 'chronobank': 10624352, 'arcblock': 106425130.83333333, 'steem-dollars': 42117429.0, 'audiocoin': 4299683, 'inflationcoin': 1536918.0, 'kcash': 0, 'acoin': 120206.0, 'casinocoin': 44016307, 'zencash': 131134937.0, 'batcoin': 0, 'droxne': 505151, 'futurexe': 2285.25, 'adcoin': 855136, 'xgox': 3976936.25, 'davorcoin': 0.0, 'hitcoin': 1067517.0, 'bittokens': 768956, 'chatcoin': 0.0, 'mineum': 269230, 'dovu': 7478493.5, 'tellurion': 0.0, 'storiqa': 20273105.458333332, 'leviar': 480186.25, 'maxcoin': 6090373.0, 'encrypgen': 16126858.333333334, 'dynamic': 11272498, 'tron': 4751139354.5, 'true-chain': 18596031.25, 'district0x': 65523600, 'rimbit': 562762, 'aidoc': 29021334.75, 'nano': 1019797126.0, 'blockcat': 6475242.75, 'sphere': 10404553.0, 'carvertical': 23392654.5, 'prime-xi': 240198.0, 'c-bit': 267385, 'global-tour-coin': 172435.75, 'btctalkcoin': nan, 'mintcoin': 11513859.0, 'bitsend': 14097375, 'nexium': 14487232.0, 'vibe': 61240576.25, 'globalcoin': 209784, 'turbocoin': 0.0, 'renos': 1167135.0, 'decision-token': 39497022.75, 'stakenet': 26521706.833333332, 'aces': 0, 'coinonat': 73889, 'ugain': 0.0, 'helleniccoin': 300624.0, 'suncontract': 26614805.0, 'animation-vision-cash': 0, 'bigup': 578587.0, 'iconomi': 148481503.0, 'zurcoin': 203194.0, 'olympus-labs': 15230475.333333334, 'exclusivecoin': 6190384, 'primulon': 0, 'goldmint': 4947226.25, 'senderon': 512480.5, 'cream': 814281, 'reftoken': 4278905.75, 'bitcedi': 60863, 'linx': 4161054.5, 'singularitynet': 112847926.75, 'shekel': 1088271.1666666667, 'zetamicron': 58281, 'asch': 55551929, 'internet-node-token': 59983899.916666664, 'futcoin': 0.0, 'pivx': 327448975.0, 'tokencard': 43261004.0, 'bitpark-coin': 879067, 'operand': 0.0, 'patientory': 13634600.0, 'happycoin': 703212, 'namocoin': 0.0, 'zero': 3878329.0, 'leverj': 6704091.416666667, 'gatcoin': 7295639.583333333, 'blocktix': 25873800, 'life': 5755348.5, 'micromoney': 3244882.0833333335, 'kilocoin': 9830033.0, 'monero-original': 0.0, 'sharpe-platform-token': 4069198.0833333335, 'digitalnote': 98546388, 'intelligent-trading-foundation': 451756.5, 'quark': 3679046.0, 'spankchain': 55426089.0, 'bnrtxcoin': 76922, 'storm': 272794804.3333333, 'alphabitcoinfund': 0, 'energo': 26716436.75, 'all-sports': 27370946.5, 'donationcoin': 0, 'bitcrystals': 9049889.0, 'kubera-coin': 521283.5833333333, 'kekcoin': 2024101, 'minexcoin': 59802090.5, 'jobscoin': 20308, 'wink': 0.0, 'maker': 638035787.0, 'bitbay': 84002003.0, 'medibloc': 102449142.0, 'mktcoin': 0.0, 'ormeus-coin': 29383922.75, 'cardano': 8062152219, 'iocoin': 28298175.0, 'betacoin': 0, 'voisecom': 8428538.5, 'coimatic-2': 8873, 'debitum-network': 16539099.375, 'dashs': 0, 'nolimitcoin': 15185039.0, 'cofound-it': 34964800, 'prizm': 17303747.0, 'tracto': 5789486.5, 'cashcoin': 336379.0, 'banyan-network': 19708841.541666668, 'mithril': 302703834.3333333, 'hicoin': 125634.0, 'achain': 155074151, 'wavesgo': 333269.0, 'rock': 40129030.583333336, 'rhfcoin': 0.0, 'putincoin': 3053844.0, 'birds': 0, 'insurepal': 9087368.583333334, 'coinfi': 12744507, 'game': 138584615.5, 'aidcoin': 7361157, 'super-bitcoin': 0.0, 'enigma': 115307.0, 'onix': 335559.25, 'energycoin': 14361247, 'superior-coin': 0.0, 'dappster': 46933, 'teslacoin': 2460859.0, 'starbase': 5138448.166666667, 'bubble': 0.0, 'swarm-fund': 17746228.416666668, 'theresa-may-coin': 163798.5, 'parkbyte': 166801.0, 'signatum': 252442.0, 'spectre-dividend': 32324148.583333332, 'decentraland': 153672457, 'electrifyasia': 50470540.75, 'universalroyalcoin': 0.0, 'revolvercoin': 253004.0, 'blockv': 94574454.25, 'oceanlab': 1491387.0, 'curriculum-vitae': 0.0, 'uniform-fiscal-object': 3791251.0, 'nem': 3725603999.0, 'shield-xsh': 36077702.0, 'iethereum': 673150.25, 'augur': 461441200, 'smartcash': 166836600.75, 'bitvolt': 22708.0, 'b3coin': 6308334.416666667, 'daneel': 0.0, 'carboncoin': 2854480, 'iostoken': 425223540.0, 'nubits': 5743792.0, 'ion': 64198392.0, 'startercoin': 3975869.2083333335, 'vector': 31744.0, 'kucoin-shares': 359344990.5, 'president-trump': 0.0, 'biobar': 15358, 'qbao': 24703766.583333332, 'netcoin': 830182.0, 'independent-money-system': 197399.0, 'experience-points': 57871545.0, 'tao': 15364382.0, 'mao-zedong': 496487.0, 'skincoin': 1803376.25, 'nexxus': nan, 'comet': 113612, 'digitalcoin': 867182, 'sense': 2708080.0833333335, 'lunyr': 32999238.0, 'digipulse': 4604341.666666667, 'first-bitcoin': 0, 'vertcoin': 106476397.0, 'sociall': 3326670.5, 'gpu-coin': 2042192.0, 'octoin-coin': 863305.4583333334, 'zetacoin': 1010095.0, 'huntercoin': 3574345.0, 'firecoin': 164009.0, 'altcoin-alt': 2854463, 'tekcoin': 507040.0, 'kzcash': 2665168.0, 'granitecoin': 0, 'dimcoin': 16588508, 'ixcoin': 3056837.0, 'eos': 12050728305, 'bibox-token': 78946448, 'naga': 42708989.916666664, 'coin': 358168, 'bitcoin-plus': 6510411, 'ronpaulcoin': 351773.0, 'apex': 103530248.95833333, 'dash': 3688829610, 'bitquark': 136040, 'arbitragect': 688772.0833333334, 'clams': 15109606, 'hshare': 457014000, 'fintrux-network': 10040025.802083334, 'sexcoin': 3153352.0, 'slevin': 60885.25, 'btcmoon': 0.0, 'ratecoin': 378467, 'president-johnson': 0, 'piggycoin': 896612, 'letitride': 38677.0, 'happy-creator-coin': 0.0, 'gobyte': 12935022, 'wings': 56793089, 'tokenbox': 2714708.75, 'flash': 21148762.5, 'wi-coin': 0.0, 'tomochain': 58308197.166666664, 'clearcoin': 9960146.666666666, 'oceanchain': 0.0, 'paypeer': nan, 'bullion': 2665106, 'internet-of-things': 0.0, 'revain': 195407570.25, 'bitrent': 42438078.916666664, 'legends-room': 1357747.5, 'locicoin': 7526105.083333333, 'real': 3739740.0, 'dcorp': 4583196.0, 'ecocoin': 211176, 'ipchain': 0.0, 'lympo': 25523397, 'swing': 224165.0, 'nimiq': 16360286.25, 'mobilecash': 0.0, 'bitcoin-green': 13865815.916666666, 'linda': 15985010, 'read': 0.0, 'streamr-datacoin': 78044427.25, 'rentberry': 12614201.791666666, 'flappycoin': 1097448, 'peculium': 0.0, 'chaincoin': 2814825, 'kin': 206547328.75, 'pascal-coin': 19857484.0, 'eroscoin': 8218109.416666667, 'einsteinium': 77888024, 'gapcoin': 311516, 'islacoin': 397262.0, 'minex': 0, 'copytrack': 4433894.875, 'siacoin': 742196300.0, 'icos': 16443932.5, 'insanecoin-insn': 966555, 'aidos-kuneen': 0.0, 'op-coin': 219142.08333333334, 'ethereum-movie-venture': 1857965.0, 'interplanetary-broadcast-coin': 15079540.208333334, 'boolberry': 12322208, 'high-performance-blockchain': 117065201, 'blakestar': 0, 'loopring': 474697887, 'mediccoin': 9250321, 'sibcoin': 22675420.0, 'utrust': 56856592.25, 'quotient': 0.0, 'iticoin': 618160.0, 'soilcoin': 113364.0, 'version': 1346280.25, 'embers': 1093910.0, 'maggie': 0.0, 'sether': 4922061.541666667, 'atmcoin': 0, 'status': 519416896.75, 'curecoin': 10327323, 'polybius': 18265159, 'zozocoin': 74584.0, 'ethereumcash': 1116001.0, 'axpire': 22338281, 'taas': 49154353.0, 'sumokoin': 5473989.25, 'bitcoin-god': 0, 'agrello-delta': 24589871.25, 'tristar-coin': 2773.0, 'deuscoin': 1498758.0, 'sureremit': 18271425.0, 'crowdcoin': 1004722, 'global-cryptocurrency': 1945975.0, 'educare': 30565359, 'bitcoin-21': 147792, 'uttoken': 0.0, 'storjcoin-x': 0.0, 'neo': 4523675000, 'leafcoin': 1989018.0, 'jesus-coin': 1599809.5833333333, 'ripple': 32024843514.0, 'thecreed': 0.0, 'hoqu': 8689057.375, 'zap': 18689219.166666668, 'asiacoin': 8296612, 'gbcgoldcoin': 26927, 'entcash': 0.0, 'wandx': 1773208.25, 'rivetz': 10371393, 'raiden-network-token': 106141790.25, 'counterparty': 47828167, 'ongsocial': 2520904.9166666665, 'uro': 60001.0, 'xios': 224508.75, 'xenon': 7464495.0, 'cryptonite': 600701, 'spacecoin': 252235.0, 'pressone': 0.0, 'nxt': 218599169.0, 'darsek': 303818, 'power-ledger': 207821956.75, 'commerceblock': 14775151.729166666, 'ubiq': 94617897.0, 'mindcoin': 57188.0, 'veritaseum': 250328673.0, 'torcoin-tor': 27161.0, 'medishares': 59103381, 'investfeed': 10806524.0, 'binance-coin': 1748743354, 'mobilego': 40343587.5, 'chronologic': 2401502, 'vezt': 4510048.0, 'get-protocol': 8413184.166666666, 'hobonickels': 857896.0, 'privatix': 6311862.0, 'animecoin': 0, 'matryx': 21290728.5, 'cashme': 0, 'w3coin': 0.0, 'blockcdn': 0, 'realchain': 0.0, 'cthulhu-offerings': 45216, 'belugapay': 0.0, 'gold-reward-token': 0.0, 'russiacoin': 4922586.0, 'crystal-clear': 849027.25, 'waves-community-token': 11042824.0, 'nitro': 0.0, 'tierion': 56054348, 'shacoin': 0, 'golos-gold': 0.0, 'everus': 14348667.0, 'scryinfo': 75057097.5, 'bitswift': 3586729, 'veltor': 45585.0, 'bounty0x': 14174509, 'x2': 0.0, 'trustplus': 3398997.0, 'magnetcoin': 0.0, 'strikebitclub': 0.0, 'atn': 10603115, 'teslacoilcoin': 0.0, 'kushcoin': 492131.0, 'decent': 44529657.25, 'tagrcoin': 27486.0, 'virtualcoin': nan, '1world': 5052353.25, 'crave': 2806574.0, 'covesting': 22218743.666666668, 'mojocoin': 637122.0, 'ethos': 262739349.75, 'cryptoescudo': 237777, 'trezarcoin': 3251287.25, 'luxcoin': 20674915.25, 'topchain': 0.0, 'zilliqa': 664777283.9166666, 'phantomx': 203680.91666666666, 'topaz': 0.0, 'p7coin': 9887.0, 'vice-industry-token': 0.0, 'ugchain': 29886121.083333332, 'wild-crypto': 961525.5, 'crypto': 396762, 'triggers': 52171242.0, 'pluscoin': 1058183.0, 'datawallet': 24845770.5, 'quantum-resistant-ledger': 69895410.0, 'dent': 130428609.25, 'farad': 2625004.25, 'coinmeet': 0.0, 'mincoin': 83330.0, 'numuscash': 0.0, 'dotcoin': 7064921, 'allsafe': 90158, 'luckchain': 4034125.0, 'pluton': 12194687.0, 'ark': 326591388, 'burst': 47100349, 'hush': 5673908.0, 'bitcore': 120075759, 'sharex': 6033852.666666667, 'fonziecoin': 0, 'santiment': 120716614.0, 'avatarcoin': 0, 'target-coin': 11585071.0, 'ebittree-coin': 1088.0, 'fastcoin': 619094.0, 'cloud': 0.0, 'milocoin': 42195.0, 'yacoin': 170456.0, 'cube': 98798746.70833333, 'golos': 12259146.0, 'litebitcoin': 140185.5, 'aseancoin': 0, 'blakecoin': 395854, 'funfair': 238479659, '0x': 574926806.0, 'manna': 478032.0, 'piplcoin': 3487602.5, 'centra': 1223972, 'bitcoinz': 7274038, 'interzone': 80353, 'bigone-token': 0, 'regalcoin': 1098848.25, 'ost': 74970349.41666667, 'psilocybin': nan, 'hollywoodcoin': 10725166, 'elastic': 32374407.5, 'i0coin': 1016624.0, 'speedcash': 320737.0, 'comsa-xem': 0, 'delphy': 47013249.25, 'moeda-loyalty-points': 23111690.25, 'sixeleven': 170249.0, 'block-array': 17178601.083333332, 'alphacat': 36949174, 'segwit2x': 0.0, 'substratum': 281404762.25, 'hyperstake': 5978564.0, 'osmiumcoin': 24926.0, 'hexx': 6965186.0, 'basic-attention-token': 464163000, 'gambit': 11691032.0, 'phore': 27107207.0, 'blox': 41342041.0, 'anryze': 0, 'innova': 2320299.5, 'centurion': 704894, 'ico-openledger': 854462, 'martexcoin': 2458736.0, 'smileycoin': 2409035.0, 'ebtcnew': 2606149.25, 'globaltoken': 211949, 'ethereum-dark': 210924, 'graft': 3258533.125, 'ink-protocol': 9679032.541666666, 'nekonium': 238580.33333333334, 'stockchain': 0.0, 'enjin-coin': 114376578.0, 'zclassic': 80803578.0, 'billarycoin': 91740, 'digital-developers-fund': 398059, 'digibyte': 346880930, 'dadi': 24710628.083333332, 'neblio': 169210327.0, 'goodomy': 4075073, 'marscoin': 296121, 'hempcoin-hmp': 128466.0, 'metal-music-coin': 99933.0, 'blocklancer': 2156303.75, 'escoro': 0.0, 'solarcoin': 16085959.0, 'social-send': 5700100, 'nexus': 140958951.0, 'leviarcoin': 492593, 'jiyo': 3484279.0, 'artbyte': 8631919, 'litecoin-cash': 105313152.45833333, 'maidsafecoin': 161743589.0, 'paypie': 111174937.5, 'signal-token': 9127860.583333334, 'win-coin': 0, 'petrodollar': 985310.0, 'fortuna': 31580348, 'zonecoin': 32940.0, 'socialcoin-socc': 11579.5, 'worldcoin': 1721239.0, 'uptoken': 31431494.458333332, 'arcticcoin': 1438132, 'sequence': 9950628.0, '2give': 5131623, 'rubies': 211072.0, 'axiom': 0, 'sharkcoin': 0.0, 'wearesatoshi': 0.0, 'reddcoin': 220625768.0, 'pabyosi-coin-special': 0.0, 'network-token': nan, 'sportyco': 8068815.916666667, 'sterlingcoin': 262093.0, 'healthywormcoin': 109710, 'atomic-coin': 238043, 'lisk': 1186711043.0}


In [56]:
df = pd.DataFrame([[k, market_cap[k]] for k in market_cap], columns=['name', 'values'])

In [58]:
sorted_market_cap = sorted([[k, market_cap[k]] for k in market_cap.keys()], key=lambda x: x[1], reverse=True)

In [65]:
df = df.sort_values(by='values', ascending=False)


In [67]:
df['sort'] = range(len(df.index))


In [83]:
df[df['name'] == 'bitcoin'].iloc[0, 2]

0